In [12]:
import ee
import zipfile
import geopandas as gpd
import ee
import pandas as pd

ee.Initialize(project='ee-caiosimplicioarantestes1')
ee.Authenticate

<function ee.Authenticate(authorization_code: Optional[str] = None, quiet: Optional[bool] = None, code_verifier: Optional[str] = None, auth_mode: Optional[str] = None, scopes: Optional[Sequence[str]] = None, force: bool = False) -> Optional[bool]>

In [13]:
shapefile_path = "./shape/area.kml"

aoi = gpd.read_file(shapefile_path)

# Dissolve geometries if multiple exist
aoi = aoi.dissolve()

geometry = aoi.geometry.iloc[0]

# Convert to GeoJSON and remove third dimension
geojson = geometry.__geo_interface__
geojson['coordinates'] = [
    [coord[:2] for coord in ring] if geojson['type'] == 'Polygon' else
    [[coord[:2] for coord in ring] for ring in polygon]
    for ring in geojson['coordinates']
]

# Create Earth Engine FeatureCollection
aoi = ee.FeatureCollection([ee.Feature(ee.Geometry(geojson))])


In [14]:
# Define the start and end dates for filtering the image collection

import pandas as pd
from datetime import datetime
inicio = '2004-01-11'
final = '2005-01-01'
nuvem = 50

# Load the Landsat 5 image collection and filter by date, location, and cloud coverage
landsat = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2') \
    .filterDate(inicio, final) \
    .filterBounds(aoi) \
    .filter(ee.Filter.lt('CLOUD_COVER', nuvem)) \
    .map(lambda image: image.set('date', image.date().format('YYYY-MM-dd')))

# Get the number of images in the collection
count = landsat.size().getInfo()
print(f"Number of images in collection: {count}")

Number of images in collection: 12


In [16]:
import requests
from requests import exceptions
import os

# Define the download URL
try:
    # Ensure 'scale' is set to the native resolution (e.g., 30 for most Landsat bands)
    scale = 30  # Or 15 for panchromatic band, if applicable
    url = first_image.getDownloadURL({
        'scale': scale,
        'region': aoi.geometry().bounds().getInfo()['coordinates'],
        'filePerBand': False  # Keep bands in a single file
    })
    print(f"Download URL: {url}")
except Exception as e:
    print(f"Error generating download URL: {e}")
    exit()

# Define the path to save the downloaded file
output_path = './landsat_image.tif'

# Download the file from the URL and save it to the specified path
try:
    with requests.get(url, stream=True) as response:
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
        total_length = response.headers.get('content-length')

        with open(output_path, 'wb') as file:
            if total_length is None:  # no content length header
                file.write(response.content)
            else:
                dl = 0
                total_length = int(total_length)
                for chunk in response.iter_content(chunk_size=8192):  # Adjust chunk size as needed
                    dl += len(chunk)
                    file.write(chunk)
                    done = int(50 * dl / total_length)
                    print(f"\r[{'=' * done}{' ' * (50-done)}]", end="")
        print() # newline after progress bar

    # File size check
    file_size = os.path.getsize(output_path)
    if total_length is not None and file_size != total_length:
        print(f"Warning: Downloaded file size ({file_size} bytes) does not match expected size ({total_length} bytes).")

    print(f"File downloaded successfully and saved to {output_path}")

except exceptions.RequestException as e:
    print(f"Failed to download file due to a request exception: {e}")
except IOError as e:
    print(f"Failed to write file to disk: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

#  **************************************************************************
#  REMEMBER:  The quality of the downloaded image critically depends on the
#  Earth Engine processing (cloud masking, compositing, etc.) that you
#  perform *before* generating the download URL. This script *only* handles
#  the download itself.
#  **************************************************************************

Download URL: https://earthengine.googleapis.com/v1/projects/ee-caiosimplicioarantestes1/thumbnails/a47940f1e3faf19e1adfa4cc4e93023d-f0c42a8782af73d6ab551f38b2a56387:getPixels
[==================================================]
File downloaded successfully and saved to ./landsat_image.tif
